---

# Lib

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MultiLabelBinarizer

from preprocessing.clean import vn_text_clean
from preprocessing.tokenize import vn_word_tokenize
from preprocessing.remove_stopwords import remove_stopwords
from utils import parse_label, matrix_labels

[nltk_data] Downloading package punkt to C:\Users\DANG
[nltk_data]     CUONG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


---

# Read file

In [2]:
# Train
df_train = pd.read_csv("../data/raw/train.csv")
df_train.columns = df_train.columns.str.strip().str.lower().str.replace(' ', '_')

In [3]:
# Validation
df_val = pd.read_csv("../data/raw/val.csv")
df_val.columns = df_val.columns.str.strip().str.lower().str.replace(' ', '_')

In [4]:
df_train = df_train[["comment", "label"]].copy()
df_val = df_val[["comment", "label"]].copy()

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  908 non-null    object
 1   label    908 non-null    object
dtypes: object(2)
memory usage: 14.3+ KB


In [6]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  99 non-null     object
 1   label    99 non-null     object
dtypes: object(2)
memory usage: 1.7+ KB


---

# Clean text

In [7]:
df_train["comment"] = df_train["comment"].apply(lambda x: vn_text_clean(x))
df_val["comment"] = df_val["comment"].apply(lambda x: vn_text_clean(x))

In [8]:
df_train.iloc[0, 0]

'mẫu này đẹp và hiện đại hơn mặt đồng hồ trung tâm đã làm âm vào mặt taplo chứ không như cái ipad dựng đứng tạo ra khoảng trống phía sau lái xe có để gì ở sau sẽ không nhìn thấy'

In [9]:
df_val.iloc[0,0]

'mẫu này đẹp và hiện đại hơn mặt đồng hồ trung tâm đã làm âm vào mặt taplo chứ không như cái ipad dựng đứng tạo ra khoảng trống phía sau lái xe có để gì ở sau sẽ không nhìn thấy'

---

# Tokenize

In [10]:
df_train["comment"] = df_train["comment"].apply(vn_word_tokenize, method="underthesea")
df_val["comment"] = df_val["comment"].apply(vn_word_tokenize, method="underthesea")

In [11]:
df_train.iloc[0, 0]

'mẫu này đẹp và hiện_đại hơn mặt đồng_hồ trung_tâm đã làm âm vào mặt taplo chứ không như cái ipad dựng_đứng tạo ra khoảng trống phía sau lái_xe có để gì ở sau sẽ không nhìn thấy'

In [12]:
df_val.iloc[0, 0]

'mẫu này đẹp và hiện_đại hơn mặt đồng_hồ trung_tâm đã làm âm vào mặt taplo chứ không như cái ipad dựng_đứng tạo ra khoảng trống phía sau lái_xe có để gì ở sau sẽ không nhìn thấy'

---

# Remove stopwords

In [13]:
# Backup comment for remove stopwords
df_train["comment_backup"] = df_train["comment"].copy()
df_val["comment_backup"] = df_val["comment"].copy()

In [14]:
def remove_stopwords_wrapper(df: pd.DataFrame) -> pd.DataFrame:
    # Remove stopwords with fallback if result becomes empty/NaN
    post = df["comment"].apply(remove_stopwords)

    # Normalize empties to NaN to enable fallback
    post = post.replace("", np.nan)

    print(post.isnull().sum())

    # Fallback to backup text where stopword removal produced NaN
    fallback = post.fillna(df["comment_backup"])

    df["comment"] = fallback
    
    return df

In [15]:
df_train = remove_stopwords_wrapper(df_train)
df_val = remove_stopwords_wrapper(df_val)

2
0


In [16]:
df_train.iloc[0, 0]

'mẫu đẹp hiện_đại mặt đồng_hồ trung_tâm âm mặt taplo ipad dựng_đứng trống lái_xe'

In [17]:
df_val.iloc[0, 0]

'mẫu đẹp hiện_đại mặt đồng_hồ trung_tâm âm mặt taplo ipad dựng_đứng trống lái_xe'

In [18]:
# Remove backup columns
df_train.drop(columns="comment_backup", axis=1, inplace=True)
df_val.drop(columns="comment_backup", axis=1, inplace=True)

---

# Save file

In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  908 non-null    object
 1   label    908 non-null    object
dtypes: object(2)
memory usage: 14.3+ KB


In [20]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  99 non-null     object
 1   label    99 non-null     object
dtypes: object(2)
memory usage: 1.7+ KB


In [21]:
df_train.to_csv("../data/processed/train.csv", index=False)
df_val.to_csv("../data/processed/val.csv", index=False)